In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys

url="https://portal.hanyang.ac.kr/sso/lgin.do"
browser=webdriver.Chrome()
browser.get(url)

time.sleep(3)

# 바로가기 클릭
browser.find_element_by_class_name("goPortal").click()
time.sleep(2)
# 로그인하기
login=browser.find_element_by_id("userId")
login.send_keys("rladnjsry837")
pwd=browser.find_element_by_id("password")
pwd.send_keys("rladnjsry23")
pwd.send_keys(Keys.ENTER)
time.sleep(2)
# 비밀번호 다음에 변경 클릭
browser.find_element_by_id("btn_cancel").click()
time.sleep(3)

# 교과목 포트폴리오로 들어가기
browser.find_element_by_link_text('학사행정').click()
time.sleep(2)
browser.find_element_by_link_text('교과목포트폴리오').click()
time.sleep(2)

# 서울로 선택
browser.find_element_by_id('cbCampus').click()
time.sleep(2)
browser.find_element_by_xpath('//*[@id="cbCampus"]/option[2]').click() # 서울대학 선택
time.sleep(2)
browser.find_element_by_id('cbHakgwa').click()
time.sleep(2)
browser.find_element_by_xpath('//*[@id="cbHakgwa"]/option[2]').click()
time.sleep(2)
# 조회 클릭
browser.find_element_by_id('btn_Find').click()
time.sleep(5)


## 학수번호와 과목명 긁어오기

In [ ]:
# beautifulsoup으로 접근
import requests
from bs4 import BeautifulSoup
soup=BeautifulSoup(browser.page_source, "lxml")

# 학수번호 리스트 만들기
time.sleep(2)
tbody=soup.find_all("tbody")[1]
trs=tbody.find_all('tr')
time.sleep(2)
haksuNo=[tr.find('td', attrs={'id':'haksuNo'}).get_text() for tr in trs]

# 학과명 리스트 만들기
gwamokNm=[tr.find('td', attrs={'id':'gwamokNm'}).get_text() for tr in trs]

# 설강 기간 리스트 만들기
startEndYearterm=[tr.find('td', attrs={'id':'startEndYearterm'}).get_text() for tr in trs]

## 리스트들을 dataframe 형태로 정리

In [7]:
import pandas as pd

dic = {
    '학수번호' : haksuNo,
    '학과명' : gwamokNm,
    '설강기간' : startEndYearterm
}
df=pd.DataFrame(dic)
df

,학수번호,학과명,설강기간
0,AIX0001,인공지능과학자-딥러닝,2018/2 ~ 2023/2
1,AIX0002,AI+X:인공지능,2019/2 ~ 2023/2
2,AIX0003,AI+X:딥러닝,2019/2 ~ 2023/2
3,AIX0004,AI+X:R-Py컴퓨팅,2019/2 ~ 2023/2
4,AIX0005,AI+X:머신러닝,2019/2 ~ 2023/2
...,...,...,...
1156,VEN3019,창업실무:쉽게배우는모바일앱개발,2022/1 ~ 2023/2
1157,VEN4001,창업실습:캡스톤디자인,2015/2 ~ 2023/2
1158,VEN4003,블록체인과비즈니스모델,2018/2 ~ 2023/2
1159,VEN6001,스타트업토크콘서트,2015/2 ~ 2023/2


In [9]:
len(df) # 설강기간 필터 전 강좌 수

# 2022년 1학기에 설강돼 있는 학과를 골라야 하기 때문에 필터
filt=df['설강기간'].str.contains('2023'or'2022'or'2021')
print(len(df[filt])) # 필터 후 강좌 수
df=df[filt]

# 엑셀로 저장
df.to_excel('강좌.xlsx')

# 엑셀로 확인해보니 모두 설강기간이 2023/2 까지이다. 그런데 나는 평점비율전적이 나와있지 강의는 필요가 없으므로 2022년 1학기에 신설되는 강좌를 다시 걸러준다
filt=df['설강기간'].str.contains('2022')
print(len(df[-filt])) # 이때, 2022를 제외한 강좌를 걸러내야 하므로 역을 걸어줘야 하는 것 잊지말자
df=df[-filt]

# 다시 엑셀로 저장
df.to_excel('강좌.xlsx')

703
695


## 2021-2 평점 있는지 긁어오기+아예 평점이 없는지 긁어오기 (전 학기 평점이 없으면 데이터가치가 없다)

### time.sleep을 넣지 않으면 코드를 찾지를 못한다. 이것만 6시간 걸렸다. 코드가 찾아지지 않거나 텍스트를 긁어오지 못하면 상위에서 하위로 차례대로 내려가고 time.sleep 넣자.

In [ ]:
year=[]
n=0
while n<67:
    browser.switch_to.window(browser.window_handles[0])
    browser.find_element_by_xpath('//td[contains(text(), "' + df.iloc[n]['학수번호'] + '")]').click() ####### 기억!!!!!!!!!!!!!!!!
    time.sleep(2)
    browser.switch_to.window(browser.window_handles[1])
    time.sleep(2)
    browser.find_element_by_xpath('//*[@id="sungJukBuyoAvgStateDiv"]/div[1]/span/a/img').click() # +클릭
    time.sleep(2)
    tbody=browser.find_element_by_xpath('//*[@id="sungJukBuyoAvgStateGridTbl"]/tbody') # tbody
    time.sleep(2)
    tr=tbody.find_element_by_tag_name('tr') # tr
    time.sleep(2)
    
    if tr.find_element_by_tag_name('td').text == '조회를 하지 않았거나 조회된 데이터가 없습니다.':
        id='nan'
        time.sleep(2)
    else:
        id=tr.find_element_by_id('suupYear').text
        time.sleep(2)
    
    year.append(id)
    
    n+=1
    
print(year)

In [ ]:
n=67
while n<100:
    browser.switch_to.window(browser.window_handles[0])
    browser.find_element_by_xpath('//td[contains(text(), "' + df.iloc[n]['학수번호'] + '")]').click() ####### 기억!!!!!!!!!!!!!!!!
    time.sleep(2)
    browser.switch_to.window(browser.window_handles[1])
    time.sleep(2)
    browser.find_element_by_xpath('//*[@id="sungJukBuyoAvgStateDiv"]/div[1]/span/a/img').click() # +클릭
    time.sleep(2)
    tbody=browser.find_element_by_xpath('//*[@id="sungJukBuyoAvgStateGridTbl"]/tbody') # tbody
    time.sleep(2)
    tr=tbody.find_element_by_tag_name('tr') # tr
    time.sleep(2)
    
    if tr.find_element_by_tag_name('td').text == '조회를 하지 않았거나 조회된 데이터가 없습니다.':
        id='nan'
        time.sleep(2)
    else:
        id=tr.find_element_by_id('suupYear').text
        time.sleep(2)
    
    year.append(id)
    
    n+=1
    
print(year)

In [ ]:
n=100
while n<len(df):
    browser.switch_to.window(browser.window_handles[0])
    browser.find_element_by_xpath('//td[contains(text(), "' + df.iloc[n]['학수번호'] + '")]').click() ####### 기억!!!!!!!!!!!!!!!!
    time.sleep(2)
    browser.switch_to.window(browser.window_handles[1])
    time.sleep(2)
    browser.find_element_by_xpath('//*[@id="sungJukBuyoAvgStateDiv"]/div[1]/span/a/img').click() # +클릭
    time.sleep(2)
    tbody=browser.find_element_by_xpath('//*[@id="sungJukBuyoAvgStateGridTbl"]/tbody') # tbody
    time.sleep(2)
    tr=tbody.find_element_by_tag_name('tr') # tr
    time.sleep(2)
    
    if tr.find_element_by_tag_name('td').text == '조회를 하지 않았거나 조회된 데이터가 없습니다.':
        id='nan'
        time.sleep(2)
    else:
        id=tr.find_element_by_id('suupYear').text
        time.sleep(2)
    
    year.append(id)
    
    n+=1
    
print(year)

In [21]:
print(len(df))
print(len(year))
# 같은 거 확인

695
695


## 연도를 dataframe의 새로운 칼럼에 추가하고 2021년도가 아니면 삭제한다.

In [38]:
df['연도']=year
# 2021년도가 아니면 데이터가 가치가 없으므로 2021년도만 남겨두고 다른 셀은 모두 삭제한다.
filt=df['연도'].isin(['2021'])
df[filt]
df=df[filt]


# 엑셀로도 저장
df.to_excel('강좌.xlsx')

## 평점 긁어오기

In [7]:
df2=pd.read_excel('강좌.xlsx') ## 다음날 해서 엑셀에 저장한 것을 읽어온다
df2

,Unnamed: 0,학수번호,학과명,설강기간,연도
0,1,AIX0002,AI+X:인공지능,2019/2 ~ 2023/2,2021
1,2,AIX0003,AI+X:딥러닝,2019/2 ~ 2023/2,2021
2,3,AIX0004,AI+X:R-Py컴퓨팅,2019/2 ~ 2023/2,2021
3,4,AIX0005,AI+X:머신러닝,2019/2 ~ 2023/2,2021
4,5,AIX0006,AI와법,2020/2 ~ 2023/2,2021
...,...,...,...,...,...
380,1150,VEN3011,창업기초:문화예술산업의이해,2016/1 ~ 2023/2,2021
381,1153,VEN3014,창업실습:스타트랙1,2019/1 ~ 2023/2,2021
382,1154,VEN3017,클라우드비즈니스모델,2021/1 ~ 2023/2,2021
383,1158,VEN4003,블록체인과비즈니스모델,2018/2 ~ 2023/2,2021


In [ ]:
n=0
points=[]
while n<199:
    browser.switch_to.window(browser.window_handles[0])
    browser.find_element_by_xpath('//td[contains(text(), "' + df2.iloc[n]['학수번호'] + '")]').click() ####### 기억!!!!!!!!!!!!!!!!
    time.sleep(2)
    browser.switch_to.window(browser.window_handles[1])
    time.sleep(2)
    browser.find_element_by_xpath('//*[@id="sungJukBuyoAvgStateDiv"]/div[1]/span/a/img').click() # +클릭
    time.sleep(2)
    tbody=browser.find_element_by_xpath('//*[@id="sungJukBuyoAvgStateGridTbl"]/tbody') # tbody
    time.sleep(2)
    tr=tbody.find_element_by_tag_name('tr') # tr
    time.sleep(2)
    
  
    id=tr.find_element_by_id('dsgIwAvg').text
    time.sleep(2)
    
    points.append(id)
    
    n+=1
    
print(points)

In [12]:
points=['4.33', '3.01', '3.2', '4.21', '3.78', '3.79', '3.42', '3.4', '3.49', '4.06', '3.85', '3.91', '3.94', '3.61', '3.95', '3.46', '3.7', '3.81', '4', '3.64', '3.96', '3.95', '4.23', '3.72', '4.3', '4.32', '4.41', '4.2', '3.89', '3.8', '3.64', '3.59', '3.53', '3.46', '3.45', '3.85', '3.73', '3.56', '3.68', '3.85', '0', '3.78', '4.15', '3.25', '3.83', '0', '3.29', '4.1', '4.21', '3.65', '3.7', '3.33', '3.73', '3.83', '3.78', '3.41', '3.39', '3.71', '3.74', '3.61', '3.84', '3.2', '3.61', '3.92', '3.76', '3.94', '3.85', '3.6', '3.82', '4.33', '3.53', '4.42', '3.75', '3.6', '3.76', '3.84', '3.93', '3.86', '3.95', '3.79', '3.92', '3.44', '3.61', '3.71', '4.1', '3.73', '3.44', '3.52', '3.79', '4', '0', '4.43', '3.47', '4.46', '4.41', '3.9', '4.17', '3.93', '3.73', '4.32', '3.71', '3.64', '4.39', '4.5', '3.75', '4.2', '3.29', '4.5', '4.43', '3.62', '3.35', '4.5', '3.49', '4.11', '4.5', '3.69', '3.09', '3.68', '3.66', '3.61', '3.16', '3.45', '4.49', '3.59', '3.54', '0', '3.77', '3.69', '3.21', '3.51', '0', '0', '0', '0', '0', '3.68', '3.76', '3.74', '3.57', '3.46', '3.92', '0', '3.81', '0', '3.5', '3.85', '3.35', '3.71', '3.71', '3.58', '0', '0', '3.53', '3.42', '3.81', '4.01', '3.71', '3.68', '0', '0', '0', '3.87', '3.66', '3.54', '3.51', '3.65', '3.89', '3.68', '3.64', '3.92', '4.02', '4.07', '3.76', '3.91', '0', '0', '3.29', '0', '4.42', '3.91', '3.81', '4.18', '4.3', '4.22', '0', '3.74', '3.29', '0', '3.75', '3.98', '4.24', '4.25', '3.59', '4.33', '4.19', '4.32', '0', '0', '4.2']
len(points)

199

In [ ]:
n=199
while n<len(df2):
    browser.switch_to.window(browser.window_handles[0])
    browser.find_element_by_xpath('//td[contains(text(), "' + df2.iloc[n]['학수번호'] + '")]').click() ####### 기억!!!!!!!!!!!!!!!!
    time.sleep(2)
    browser.switch_to.window(browser.window_handles[1])
    time.sleep(2)
    browser.find_element_by_xpath('//*[@id="sungJukBuyoAvgStateDiv"]/div[1]/span/a/img').click() # +클릭
    time.sleep(2)
    tbody=browser.find_element_by_xpath('//*[@id="sungJukBuyoAvgStateGridTbl"]/tbody') # tbody
    time.sleep(2)
    tr=tbody.find_element_by_tag_name('tr') # tr
    time.sleep(2)
    
  
    id=tr.find_element_by_id('dsgIwAvg').text
    time.sleep(2)
    
    points.append(id)
    
    n+=1
    
print(points)

In [21]:
len(points) # 개수 확인

df3=pd.read_excel('강좌.xlsx')
df3['평점']=points

df3.to_excel('강좌.xlsx')

## 평점 4.4 넘는 것만 필터링

In [32]:
df3['평점']=df3['평점'].astype(float) # 문자열을 float형태로 저장하는 함수
filt=df3['평점']>4.4
df3[filt]
len(df3[filt])
df3=df3[filt]
# 이 목록은 다른 엑셀에 따로 저장
df3.to_excel('평점 4.4이상 강좌.xlsx')

df3

C:\Users\김원교\AppData\Local\Temp/ipykernel_17592/263190402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['평점']=df3['평점'].astype(float) # 문자열을 float형태로 저장하는 함수


,Unnamed: 0,학수번호,학과명,설강기간,연도,평점
26,54,CLS1023,인문클래식:문학으로읽는음식과인간,2018/1 ~ 2023/2,2021,4.41
71,151,CUL1131,4차산업혁명과비즈니스,2017/2 ~ 2023/2,2021,4.42
91,183,CUL1163,IT서비스디자인,2020/1 ~ 2023/2,2021,4.43
93,185,CUL1165,버추얼피트니스이노베이션,2020/1 ~ 2023/2,2021,4.46
94,187,CUL1167,로봇의이해,2020/1 ~ 2023/2,2021,4.41
103,198,CUL1178,IoT데이터마이닝,2020/2 ~ 2023/2,2021,4.50
107,203,CUL1183,트랜스미디어디자이닝솔루션,2021/1 ~ 2023/2,2021,4.50
108,204,CUL1184,AR/VR가상공간인터렉티브콘텐츠개발랩,2021/1 ~ 2023/2,2021,4.43
111,207,CUL1187,CEO취업멘토링,2021/2 ~ 2023/2,2021,4.50
114,211,CUL1191,물리현상과컴퓨터시뮬레이션,2021/2 ~ 2023/2,2021,4.50


# 피드백: 연도와 평점을 한꺼번에 긁어오고 걸렀으면 더 시간이 빨랐을 것 같다.